# Analysis for the floor control detection (FCD) model and competitor models

This notebook includes analyses the predictions of the FCD model and the competitor models discussed in the notebook and show how they are compared over a few measurements. It also includes some stats about the annotated floor properties, and an optimised FCD model for highest accuracy.

In [1]:
import itertools
import pathlib

from scipy import optimize as soptimize
import numpy as np
import pandas as pd

import predict_fcd
import utils.annotated_floor
import utils.iteration
import utils.path

In [2]:
%load_ext autoreload

%autoreload 2

np.random.seed(1234)

In [3]:
FEATURES_DIR = pathlib.Path('features')
PREDICTIONS_DIR = pathlib.Path('predictions')

ANALYSIS_SAMPLE_RATE = 10
SAMPLE_RATE = {
    'fcd': 50,
    'optimised_fcd': 50,
    'vad': 50,
    'full-lstm': 20,
    'partial-lstm': 20,
    'random': ANALYSIS_SAMPLE_RATE,
}
OPTIMISED_FCD_PARAMS = (0.83917969, 0.03746094)  # Can be overriden by lengthy optimisation below

# Utilities

General utilities.

In [4]:
def array_to_series(x, name, sample_rate):
    '''
    Convert a numpy array to a pandas series
    with time index.
    '''
    x = x[::sample_rate // ANALYSIS_SAMPLE_RATE]
    return pd.Series(
        x,
        index=np.arange(len(x)) / ANALYSIS_SAMPLE_RATE,
        name=name,
    )

In [5]:
def utterances_to_floor(utterances_df):
    '''
    Calculate the floor timeseries from a dataframe
    of utterances (every row has start_time, end_time,
    and participant).
    '''
    return array_to_series(
        list(
            utils.annotated_floor.gen(
                utterances_df,
                sample_rate=ANALYSIS_SAMPLE_RATE,
            )
        ),
        name='floor',
        sample_rate=ANALYSIS_SAMPLE_RATE,
    )

Random model utilities.

In [6]:
def _generate_random_model_intervals(average_floor_duration):
    floor_holder = np.random.randint(2)
    previous_timestamp = 0
    while True:
        samples = np.random.exponential(average_floor_duration, 100)
        timestamps = samples.cumsum() + previous_timestamp
        for timestamp in timestamps:
            yield {
                'start_time': previous_timestamp,
                'end_time': timestamp,
                'participant': floor_holder,
            }
            floor_holder = (floor_holder * -1) + 1
            previous_timestamp = timestamp


def calculate_random_model(average_floor_duration, part_duration):
    '''
    Calculate a random floor array with turns duration distributin
    exponentially with `average_floor_duration` as mean.
    '''
    gen = _generate_random_model_intervals(average_floor_duration)
    gen = itertools.takewhile(lambda i: i['start_time'] < part_duration, gen)
    return list(
        utils.iteration.intervals_to_values_gen(
            gen,
            sample_rate=ANALYSIS_SAMPLE_RATE,
            key='participant',
        )
    )

# Annotated floor stats

In [7]:
def annotated_floor_stats_gen():
    '''
    Calculate basic stats about the annotated floor.
    '''
    for part in utils.path.session_parts_gen(train_set=True, test_set=True):
        utterances_df = pd.read_csv(FEATURES_DIR / 'utterances' / f'{part}.csv')
        floor_intervals = list(utils.annotated_floor.utterances_to_floor_intervals_gen(utterances_df))
        floor = list(
            utils.iteration.intervals_to_values_gen(
                floor_intervals,
                sample_rate=ANALYSIS_SAMPLE_RATE,
                key='participant',
            )
        )
        yield {
            'competition_for_floor': np.isnan(floor).mean(),
            'average_floor_duration': np.mean([i['end_time'] - i['start_time'] for i in floor_intervals])
        }

In [8]:
annotated_floor_stats_df = pd.DataFrame(annotated_floor_stats_gen())
annotated_floor_stats_df.describe()

,competition_for_floor,average_floor_duration
count,30.000000,30.000000
mean,0.162378,3.527355
std,0.062455,1.364506
min,0.069196,1.974816
25%,0.130197,2.379544
50%,0.150684,3.090434
75%,0.179954,4.305084
max,0.406789,7.104639


In [9]:
# Keep the average floor duration for later, for the random model
average_floor_duration = annotated_floor_stats_df['average_floor_duration'].mean()

# Performance measures generators

Generator functions to measure the performance of a model compared to the annotated floor and annotated backhannels.

In [10]:
def accuracy(model, floor):
    '''
    For each frame with defined floor (no competition nor silence)
    yields 1 if the model and the floor agrees, 0 otherwise.
    '''
    both = pd.concat([model, floor], axis=1).dropna()
    yield from (both.iloc[:, 0] == both.iloc[:, 1]).astype(int)

In [11]:
def backchannels(model, utterances_df):
    '''
    For each backchannel yield 1 if the model report a floor
    for the partner, 0 otherwise.
    '''
    backchannels = utterances_df[utterances_df['backchannel']]
    for _, bc in backchannels.iterrows():
        bc_timestamp = bc['start_time']
        prediction_at_bc = model[bc_timestamp:].values[0]
        if prediction_at_bc:
            yield int(prediction_at_bc != bc['participant'])

In [12]:
def _floor_holder_changes(array):
    array = array[~np.isnan(array)]
    items = utils.iteration.dedup(array)
    return len(list(items)) - 1  # number of changes is number of values minus 1


def stability(model, floor):
    '''
    Ratio of actual floor changes vs. predicted floor changes.
    '''
    annotated_floor_changes = _floor_holder_changes(floor)
    model_floor_changes = _floor_holder_changes(model)
    yield annotated_floor_changes / model_floor_changes

In [13]:
def lag(model, floor):
    '''
    Yield positive lags in seconds.
    '''
    model_change = pd.Series(dict(utils.iteration.dedup(model.dropna().iteritems(), key=lambda x: x[1])))
    floor_change = pd.Series(dict(utils.iteration.dedup(floor.dropna().iteritems(), key=lambda x: x[1])))

    visited_timestamps = set()
    for timestamp, prediction in model_change.iteritems():
        previous_floors = floor_change[:timestamp]
        if not previous_floors.empty:
            current_floor_timestamp = previous_floors.index[-1]
            current_floor_value = previous_floors.values[-1]
            if (current_floor_value == prediction and current_floor_timestamp not in visited_timestamps):
                yield (timestamp - current_floor_timestamp)
            visited_timestamps.add(current_floor_timestamp)

# Optimising FCD parameters for accuracy

In [14]:
def get_negative_accuracy_from_model(params, optimisation_data):

    accuracies = []
    for rms, floor in optimisation_data:
        fcd_gen = predict_fcd.gen_from_rms(rms, *params)
        fcd = array_to_series(list(fcd_gen), name='fcd', sample_rate=SAMPLE_RATE['fcd'])
        accuracies.append(accuracy(fcd, floor))
    
    return -np.mean(accuracies)

In [15]:
optimisation_data = []
for part in utils.path.session_parts_gen(train_set=True, test_set=False):
    utterances_df = pd.read_csv(FEATURES_DIR / 'utterances' / f'{part}.csv')
    floor = utterances_to_floor(utterances_df)
    rms = np.load(FEATURES_DIR / 'FCD' / f'{part}.npy')
    optimisation_data.append((rms, floor))

**Note!** This cell takes a while to run. Feel free to comment it out as the entire notebook can be run without it. The default optimised parameters (declared at the top of the notebook) will be used in that case.

In [16]:
%%time

res = soptimize.minimize(
     get_negative_accuracy_from_model,
     [0.35, 0.1],
     args=(optimisation_data, ),
     method='Nelder-Mead',
     options={'disp': True},
)
OPTIMISED_FCD_PARAMS = res.x
res

Optimization terminated successfully.
         Current function value: -0.884932
         Iterations: 28
         Function evaluations: 63
CPU times: user 11min 24s, sys: 6.04 s, total: 11min 30s
Wall time: 11min 25s


 final_simplex: (array([[0.83917969, 0.03746094],
       [0.83917969, 0.03738281],
       [0.83921387, 0.03741699]]), array([-0.88493247, -0.88493247, -0.88493247]))
           fun: -0.8849324706878398
       message: 'Optimization terminated successfully.'
          nfev: 63
           nit: 28
        status: 0
       success: True
             x: array([0.83917969, 0.03746094])

# Comparing the models performance

In [17]:
def _part_models_stats_gen(part, average_floor_duration):
    utterances_df = pd.read_csv(FEATURES_DIR / 'utterances' / f'{part}.csv')
    floor = utterances_to_floor(utterances_df)
    rms = np.load(FEATURES_DIR / 'FCD' / f'{part}.npy')

    models = {
        'fcd': np.load(PREDICTIONS_DIR / 'FCD' / f'{part}.npy'),
        'optimised_fcd': list(predict_fcd.gen_from_rms(rms, *OPTIMISED_FCD_PARAMS)),
        'full-lstm': np.load(PREDICTIONS_DIR / 'LSTM' / f'full-{part}.npy'),
        'partial-lstm': np.load(PREDICTIONS_DIR / 'LSTM' / f'partial-{part}.npy'),
        'vad': np.load(PREDICTIONS_DIR / 'VAD' / f'{part}.npy'),
        'random': calculate_random_model(
            average_floor_duration,
            part_duration=floor.index[-1],
        ),
    }

    models_df = pd.concat(
        [array_to_series(x, name=n, sample_rate=SAMPLE_RATE[n]) for n, x in models.items()],
        axis=1,
    )

    measurement_functions_and_args = {
        backchannels: utterances_df,
        **{f: floor for f in [accuracy, stability, lag]},
    }

    for model in models:
        for f, arg in measurement_functions_and_args.items():
            for value in f(models_df[model], arg):
                yield {
                    'part': part,
                    'model': model,
                    'measurement': f.__name__,
                    'value': value,
                }


def models_stats_gen(average_floor_duration):
    '''
    Calculate the performance measure for each model accross the
    test-set.
    '''
    for part in utils.path.session_parts_gen(train_set=False, test_set=True):
        yield from _part_models_stats_gen(part, average_floor_duration)

In [18]:
models_stats_df = pd.DataFrame(models_stats_gen(average_floor_duration))

In [19]:
(
    models_stats_df
    .groupby(['model', 'measurement', 'part'])
    ['value']
    .mean()
    .groupby(['model', 'measurement'])
    .agg(['mean', 'std'])
)

mean       std
model         measurement                     
fcd           accuracy      0.864072  0.024511
              backchannels  0.889487  0.116967
              lag           0.396651  0.032862
              stability     0.856768  0.183371
full-lstm     accuracy      0.904171  0.024956
              backchannels  0.816202  0.090941
              lag           0.163419  0.089304
              stability     0.290748  0.081568
optimised_fcd accuracy      0.887575  0.014861
              backchannels  0.746855  0.174999
              lag           0.156316  0.029107
              stability     0.479205  0.122705
partial-lstm  accuracy      0.878560  0.061605
              backchannels  0.769757  0.115711
              lag           0.169148  0.120208
              stability     0.242624  0.057588
random        accuracy      0.481952  0.044649
              backchannels  0.509509  0.179864
              lag           1.757282  0.403382
              stability     0.961022  0.247827
vad           accuracy      0.764909  0.082146
              backchannels  0.476896  0.219540
              lag           0.487433  0.135832
              stability     0.365151  0.071186